# OpenDART API를 활용한 기업 정보 수집 테스트

이 노트북은 OpenDART API의 주요 기능을 테스트하여 AI Agent 개발 시 기업 정보 수집 도구(Tool)에 어떤 정보를 요청하고 받을 수 있는지, 그리고 어떻게 연동할 수 있는지를 파악하는 데 도움을 주기 위해 작성되었습니다.

**OpenDART API란?**
OpenDART는 금융감독원(FSS)에서 제공하는 전자공시시스템(DART)의 공시정보를 API 형태로 제공하여 개발자들이 다양한 서비스와 애플리케이션에서 활용할 수 있도록 지원하는 서비스입니다. 기업의 개황 정보, 재무 정보, 지분 현황, 주요 공시 내용 등을 프로그래밍 방식으로 접근할 수 있습니다.

**API 키 발급 안내:**
OpenDART API를 사용하기 위해서는 먼저 [DART 개발자 사이트](https://opendart.fss.or.kr/)에서 API 인증키를 발급받아야 합니다. 발급받은 인증키를 아래 코드의 `YOUR_OPENDART_API_KEY` 부분에 입력하거나, 환경 변수 등을 통해 안전하게 관리하는 것이 좋습니다.

## 1. 기본 설정 및 라이브러리 임포트

API 요청에 필요한 라이브러리를 임포트하고, API 키 및 기본 URL을 설정합니다.

In [1]:
## 1. 기본 설정 및 라이브러리 임포트

import requests
import json
import pandas as pd
import os
import time
import zipfile
from io import BytesIO
import xmltodict
from dotenv import load_dotenv
from IPython.display import display, Markdown

# 환경 변수 파일(.env)에서 OPEN_DART_API_KEY 로드
load_dotenv()

# 🚨 중요: 발급받은 OpenDART API 인증키를 환경 변수에서 가져옵니다.
API_KEY = os.getenv("OPEN_DART_API_KEY")
BASE_URL = "https://opendart.fss.or.kr/api"

# API_KEY 존재 여부 확인
if not API_KEY:
    print("⚠️ 환경 변수 'OPEN_DART_API_KEY'가 설정되지 않았습니다.")
    print("직접 입력하려면 아래 코드의 주석을 해제하고 API 키를 입력하세요:")
    print("# API_KEY = 'YOUR_API_KEY_HERE'")
    # API_KEY = 'YOUR_API_KEY_HERE'  # 직접 입력 시 이 줄의 주석을 해제하고 키를 입력
else:
    print("✅ OpenDART API 인증키가 성공적으로 로드되었습니다.")

✅ OpenDART API 인증키가 성공적으로 로드되었습니다.


In [2]:
## 2. 개선된 API 응답 처리 함수

def handle_api_response(response):
    """API 응답을 처리하고 JSON 데이터를 반환하거나 오류를 출력합니다."""
    if response.status_code == 200:
        try:
            data = response.json()
            if data.get("message") == "정상":  # OpenDART의 실제 응답 구조
                return data
            elif data.get("status") == "013":  # 데이터 없음
                print(f"📭 데이터 없음: {data.get('message')}")
                return None
            else:
                print(f"❌ API 오류: {data.get('status')} - {data.get('message')}")
                return None
        except json.JSONDecodeError:
            print("❌ JSON 디코딩 오류가 발생했습니다. 응답 내용:")
            print(response.text[:500])
            return None
    else:
        print(f"❌ HTTP 오류: {response.status_code}")
        print(response.text)
        return None

# 테스트용 더미 응답 확인
print("🔧 응답 처리 함수 테스트:")
class DummyResponse:
    def __init__(self, status_code, json_data, text=''):
        self.status_code = status_code
        self._json_data = json_data
        self.text = text
    def json(self):
        return self._json_data

# 정상 응답 테스트
dummy_ok = DummyResponse(200, {'message':'정상', 'data':'test'})
print("✅ 정상 응답 처리:", handle_api_response(dummy_ok) is not None)

🔧 응답 처리 함수 테스트:
✅ 정상 응답 처리: True


## 2. 기업 고유번호 조회 방법

기업 고유번호를 조회하는 두 가지 방법을 제공합니다:

1. **전체 코드 다운로드 (XML)**
   - `/corpCode.xml` 엔드포인트를 통해 모든 기업의 고유번호(ZIP 압축된 XML)를 한 번에 내려받아 처리합니다.
   - 함수: `get_all_corp_codes()`
     - ZIP 파일을 메모리에서 압축 해제하고 DataFrame으로 변환
     - 상장/비상장 기업 구분 가능

2. **개별 조회 (DataFrame 기반 검색)**
   - 미리 로드한 DataFrame에서 회사명을 부분 일치 검색하여 고유번호를 반환합니다.
   - 함수: `get_corp_code(corp_name, corp_df=None)`
     - DataFrame(`corp_df`)이 없으면 `get_all_corp_codes()`로 로드 후 캐싱
     - 회사명 부분/정확 일치 검색 및 유사 이름 제안

**함수 사용 예시:**
```python
# 전체 데이터 로드
corp_df = get_all_corp_codes()

# 개별 기업 조회
code = get_corp_code("삼성전자", corp_df)
```  

In [3]:
## 3. 전체 기업 고유번호 다운로드 (수정된 버전)

def get_all_corp_codes():
    """모든 기업의 고유번호를 가져와서 DataFrame으로 반환 (ZIP 파일 처리)"""
    print("📥 기업 고유번호 데이터를 다운로드 중...")
    
    url = f"{BASE_URL}/corpCode.xml"  # 수정: .json → .xml
    params = {'crtfc_key': API_KEY}
    
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            # ZIP 파일을 메모리에서 압축 해제
            z = zipfile.ZipFile(BytesIO(response.content))
            data_xml = z.read('CORPCODE.xml').decode('utf-8')
            
            # XML을 딕셔너리로 변환
            data_odict = xmltodict.parse(data_xml)
            data_dict = json.loads(json.dumps(data_odict))
            corp_list = data_dict.get('result', {}).get('list', [])
            
            # DataFrame으로 변환
            df = pd.DataFrame(corp_list)
            print(f"✅ 총 {len(df):,}개 기업 정보를 로드했습니다.")
            
            # 상장기업만 별도로 표시
            listed_companies = df[df['stock_code'].notna()]
            print(f"📈 상장기업: {len(listed_companies):,}개")
            print(f"📊 비상장기업: {len(df) - len(listed_companies):,}개")
            
            return df
        else:
            print(f"❌ 기업 코드 다운로드 실패: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ 기업 코드 로드 중 오류: {e}")
        return None

# 기업 고유번호 데이터 로드
corp_df = get_all_corp_codes()

# 샘플 데이터 확인
if corp_df is not None:
    print("\n📋 데이터 샘플:")
    display(corp_df.head())
    
    print("\n📈 상장기업 샘플 (종목코드가 있는 기업들):")
    listed_sample = corp_df[corp_df['stock_code'].notna()].head()
    display(listed_sample[['corp_name', 'corp_code', 'stock_code']])


📥 기업 고유번호 데이터를 다운로드 중...
✅ 총 112,558개 기업 정보를 로드했습니다.
📈 상장기업: 3,868개
📊 비상장기업: 108,690개

📋 데이터 샘플:
✅ 총 112,558개 기업 정보를 로드했습니다.
📈 상장기업: 3,868개
📊 비상장기업: 108,690개

📋 데이터 샘플:


,corp_code,corp_name,corp_eng_name,stock_code,modify_date
0,00434003,다코,Daco corporation,None,20170630
1,00430964,굿앤엘에스,"Good & LS Co.,Ltd.",None,20170630
2,00388953,크레디피아제이십오차유동화전문회사,Credipia 25th Asset Securitization Specialty L...,None,20170630
3,00179984,연방건설산업,youn bao,None,20170630
4,00420143,브룩스피알아이오토메이션잉크,"BROOKS-PRI Automation, Inc.",None,20170630



📈 상장기업 샘플 (종목코드가 있는 기업들):


,corp_name,corp_code,stock_code
1946,한빛네트,00260985,036720
1958,엔플렉스,00264529,040130
1959,동서정보기술,00358545,055000
2700,애드모바일,00231567,032600
3772,리더컴,00359614,056140


In [4]:
## 4. 기업 고유번호 조회 함수 (개선된 버전)

def get_corp_code(corp_name, corp_df=None):
    """회사명으로 기업 고유번호 조회 (개선된 버전)"""
    global corp_df_global
    
    if corp_df is None:
        if 'corp_df_global' not in globals() or corp_df_global is None:
            print("📥 기업 데이터를 먼저 로드합니다...")
            corp_df = get_all_corp_codes()
            corp_df_global = corp_df
        else:
            corp_df = corp_df_global
    
    if corp_df is None:
        return None
    
    # 회사명으로 검색 (부분 일치 포함)
    matches = corp_df[corp_df['corp_name'].str.contains(corp_name, na=False)]
    
    if len(matches) == 0:
        print(f"❌ '{corp_name}'와 일치하는 기업을 찾을 수 없습니다.")
        
        # 유사한 기업명 제안
        similar = corp_df[corp_df['corp_name'].str.contains(corp_name[:2], na=False)]
        if len(similar) > 0:
            print(f"💡 '{corp_name[:2]}'가 포함된 기업들:")
            for idx, corp in similar.head(5).iterrows():
                print(f"   - {corp['corp_name']} ({corp.get('stock_code', 'N/A')})")
        return None
        
    elif len(matches) == 1:
        corp_info = matches.iloc[0]
        print(f"✅ 기업 발견: {corp_info['corp_name']}")
        print(f"   고유번호: {corp_info['corp_code']}")
        print(f"   종목코드: {corp_info.get('stock_code', 'N/A')}")
        return corp_info['corp_code']
        
    else:
        print(f"🔍 '{corp_name}'와 일치하는 기업이 {len(matches)}개 있습니다:")
        for idx, corp in matches.head(10).iterrows():
            print(f"   - {corp['corp_name']} (코드: {corp['corp_code']}, 종목: {corp.get('stock_code', 'N/A')})")
        
        # 정확히 일치하는 기업이 있는지 확인
        exact_matches = matches[matches['corp_name'] == corp_name]
        if len(exact_matches) > 0:
            corp_info = exact_matches.iloc[0]
            print(f"✅ 정확히 일치하는 기업 선택: {corp_info['corp_name']}")
            return corp_info['corp_code']
        else:
            # 첫 번째 결과 반환
            corp_info = matches.iloc[0]
            print(f"✅ 첫 번째 결과 선택: {corp_info['corp_name']}")
            return corp_info['corp_code']

# 테스트할 회사명들
test_companies = ["삼성전자", "현대자동차", "SK하이닉스"]

print("🧪 기업 고유번호 조회 테스트:")
for company in test_companies:
    print(f"\n🔍 '{company}' 검색:")
    corp_code = get_corp_code(company, corp_df)
    if corp_code:
        print(f"   결과: {corp_code}")

🧪 기업 고유번호 조회 테스트:

🔍 '삼성전자' 검색:
🔍 '삼성전자'와 일치하는 기업이 5개 있습니다:
   - 삼성전자서비스씨에스 (코드: 01345812, 종목: None)
   - 삼성전자판매 (코드: 00252074, 종목: None)
   - 삼성전자 (코드: 00126380, 종목: 005930)
   - 삼성전자서비스 (코드: 00258999, 종목: None)
   - 삼성전자로지텍 (코드: 00366997, 종목: None)
✅ 정확히 일치하는 기업 선택: 삼성전자
   결과: 00126380

🔍 '현대자동차' 검색:
🔍 '현대자동차'와 일치하는 기업이 5개 있습니다:
   - 현대자동차써비스 (코드: 00164751, 종목: None)
   - 정왕점현대자동차 (코드: 01333297, 종목: None)
   - 현대자동차충청써비스 (코드: 00914688, 종목: None)
   - 수원현대자동차운전전문학원 (코드: 01370216, 종목: None)
   - 현대자동차 (코드: 00164742, 종목: 005380)
✅ 정확히 일치하는 기업 선택: 현대자동차
   결과: 00164742

🔍 'SK하이닉스' 검색:
✅ 기업 발견: SK하이닉스
   고유번호: 00164779
   종목코드: 000660
   결과: 00164779


## 3. 기업 개황 정보 조회 (company.json / 수정된 버전)

기업 고유번호를 이용하여 기업의 기본 현황을 조회합니다. 이 함수는 API 호출 전·후에 진행 메시지를 출력하고, DART 응답 제한을 감안하여 짧은 지연을 포함합니다.

- **API Endpoint**: `/company.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호

**함수 동작:**
1. 고유번호(`corp_code`)가 없으면 즉시 종료
2. 조회 시작 메시지 출력
3. 0.5초 지연 후 API 호출
4. 응답 JSON 처리 후 다음 정보 출력:
   - 기업명, 영문명, 종목코드, 대표자, 주소, 전화번호, 홈페이지, 설립일, 결산월
5. 추가로 주요 컬럼을 DataFrame으로 요약하여 테이블 형태로 표시

**함수 정의:**
```python
get_company_overview(corp_code)
```

**사용 예시:**
```python
# 삼성전자 개황 정보 조회
samsung_corp_code = get_corp_code("삼성전자", corp_df)
if samsung_corp_code:
    samsung_overview = get_company_overview(samsung_corp_code)
```

In [5]:
## 5. 기업 개황 정보 조회 (수정된 버전)

def get_company_overview(corp_code):
    """기업 고유번호를 사용하여 기업 개황 정보를 조회합니다."""
    if not corp_code:
        print("❌ 기업 고유번호가 없어 조회할 수 없습니다.")
        return None
    
    print(f"📊 기업 개황 정보 조회 중... (고유번호: {corp_code})")
    
    url = f"{BASE_URL}/company.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code
    }
    
    # API 호출 제한을 고려한 지연
    time.sleep(0.5)
    
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data:
        print(f"\n🏢 [{data.get('corp_name', 'N/A')}] 기업 개황 정보")
        print("=" * 50)
        
        # 주요 정보를 보기 좋게 출력
        info_dict = {
            '🏢 기업명': data.get('corp_name'),
            '🌐 영문명': data.get('corp_name_eng'),
            '📈 종목코드': data.get('stock_code'),
            '👨‍💼 대표자': data.get('ceo_nm'),
            '🏠 주소': data.get('adres'),
            '📞 전화번호': data.get('phn_no'),
            '🌐 홈페이지': data.get('hm_url'),
            '📅 설립일': data.get('est_dt'),
            '📊 결산월': data.get('acc_mt')
        }
        
        for key, value in info_dict.items():
            if value and value.strip():
                print(f"{key}: {value}")
        
        # DataFrame으로도 표시
        df_info = pd.DataFrame([data])
        key_columns = ['corp_name', 'stock_code', 'ceo_nm', 'adres', 'est_dt']
        available_columns = [col for col in key_columns if col in df_info.columns]
        
        if available_columns:
            print(f"\n📋 요약 테이블:")
            display(df_info[available_columns])
        
        return data
    else:
        print("❌ 기업 개황 정보를 가져올 수 없습니다.")
    return None

# 삼성전자 기업 개황 정보 조회 테스트
samsung_corp_code = get_corp_code("삼성전자", corp_df)
if samsung_corp_code:
    samsung_overview = get_company_overview(samsung_corp_code)

🔍 '삼성전자'와 일치하는 기업이 5개 있습니다:
   - 삼성전자서비스씨에스 (코드: 01345812, 종목: None)
   - 삼성전자판매 (코드: 00252074, 종목: None)
   - 삼성전자 (코드: 00126380, 종목: 005930)
   - 삼성전자서비스 (코드: 00258999, 종목: None)
   - 삼성전자로지텍 (코드: 00366997, 종목: None)
✅ 정확히 일치하는 기업 선택: 삼성전자
📊 기업 개황 정보 조회 중... (고유번호: 00126380)

🏢 [삼성전자(주)] 기업 개황 정보
🏢 기업명: 삼성전자(주)
🌐 영문명: SAMSUNG ELECTRONICS CO,.LTD
📈 종목코드: 005930
👨‍💼 대표자: 전영현
🏠 주소: 경기도 수원시 영통구  삼성로 129 (매탄동)
📞 전화번호: 02-2255-0114
🌐 홈페이지: www.samsung.com/sec
📅 설립일: 19690113
📊 결산월: 12

📋 요약 테이블:

🏢 [삼성전자(주)] 기업 개황 정보
🏢 기업명: 삼성전자(주)
🌐 영문명: SAMSUNG ELECTRONICS CO,.LTD
📈 종목코드: 005930
👨‍💼 대표자: 전영현
🏠 주소: 경기도 수원시 영통구  삼성로 129 (매탄동)
📞 전화번호: 02-2255-0114
🌐 홈페이지: www.samsung.com/sec
📅 설립일: 19690113
📊 결산월: 12

📋 요약 테이블:


,corp_name,stock_code,ceo_nm,adres,est_dt
0,삼성전자(주),005930,전영현,경기도 수원시 영통구 삼성로 129 (매탄동),19690113


## 4. 주요 재무정보 조회 (fnlttSinglAcnt.json / fnlttMultiAcnt.json / 수정된 버전)

기업 고유번호, 사업연도 및 보고서 코드를 사용하여 주요 재무정보를 조회합니다. 이 함수는 DART API 호출 전·후에 메시지를 출력하고, 응답 제한을 감안하여 0.5초 지연을 포함합니다.

- **API Endpoint**:
  - `/fnlttSinglAcnt.json`: 주요 단일계정
  - `/fnlttMultiAcnt.json`: 상세 다중계정
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호
  - `bsns_year`: 사업연도 (예: "2023")
  - `reprt_code`: 보고서 코드 (`11011`: 연간, `11012`: 반기, `11013`: 1분기, `11014`: 3분기)

**함수 동작:**
1. `corp_code` 유효성 확인
2. `reprt_code`→보고서명 매핑 및 조회 메시지 출력
3. 0.5초 지연 후 API 호출
4. `list` 항목을 DataFrame으로 변환
5. 주요 컬럼 요약(상위 15개) 테이블로 표시
6. 주요 재무지표(매출액, 영업이익, 당기순이익 등) 별도 추출 및 출력

**함수 정의:**
```python
get_financial_statement(corp_code, bsns_year, reprt_code="11011")
```

**사용 예시:**
```python
# 삼성전자 2023년 사업보고서(연간) 재무정보 조회
samsung_fs_2023 = get_financial_statement(samsung_corp_code, "2023", "11011")
```

In [6]:
## 6. 재무정보 조회 (수정된 버전)

def get_financial_statement(corp_code, bsns_year, reprt_code="11011"):
    """기업의 특정 연도, 보고서 종류에 따른 재무제표를 조회합니다."""
    if not corp_code:
        print("❌ 기업 고유번호가 없어 조회할 수 없습니다.")
        return None
    
    # 보고서 코드 매핑
    report_names = {
        "11013": "1분기보고서",
        "11012": "반기보고서", 
        "11014": "3분기보고서",
        "11011": "사업보고서(연간)"
    }
    
    report_name = report_names.get(reprt_code, f"보고서코드{reprt_code}")
    print(f"💰 재무정보 조회 중... ({bsns_year}년 {report_name})")
    
    url = f"{BASE_URL}/fnlttSinglAcnt.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': bsns_year,
        'reprt_code': reprt_code
    }
    
    # API 호출 제한을 고려한 지연
    time.sleep(0.5)
    
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        fs_list = data['list']
        print(f"✅ {bsns_year}년 {report_name} 재무 정보 ({len(fs_list)}개 항목)")
        
        df = pd.DataFrame(fs_list)
        
        # 주요 컬럼 확인 및 선택
        important_columns = ['sj_nm', 'account_nm', 'account_detail', 'thstrm_nm', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount']
        available_columns = [col for col in important_columns if col in df.columns]
        
        if available_columns:
            print(f"\n📊 재무정보 요약 (상위 15개 항목):")
            display_df = df[available_columns].head(15)
            display(display_df)
            
            # 주요 재무 지표 추출 (매출액, 영업이익, 당기순이익 등)
            key_accounts = ['매출액', '영업이익', '당기순이익', '총자산', '자본총계']
            print(f"\n💡 주요 재무지표:")
            for account in key_accounts:
                matched = df[df['account_nm'].str.contains(account, na=False)]
                if len(matched) > 0:
                    for _, row in matched.head(1).iterrows():
                        amount = row.get('thstrm_amount', 'N/A')
                        if amount and amount != 'N/A':
                            try:
                                amount_num = int(amount.replace(',', ''))
                                amount_formatted = f"{amount_num:,}"
                            except:
                                amount_formatted = amount
                        else:
                            amount_formatted = 'N/A'
                        print(f"   📈 {row['account_nm']}: {amount_formatted}")
        
        return fs_list
    else:
        print(f"❌ {bsns_year}년 {report_name} 재무 정보를 찾을 수 없습니다.")
        print("   💡 다른 연도나 보고서 유형을 시도해보세요.")
    return None

# 삼성전자 2023년 사업보고서(연간) 재무 정보 조회 테스트
if samsung_corp_code:
    print("\n" + "="*60)
    samsung_fs_2023 = get_financial_statement(samsung_corp_code, "2023", "11011")

# 현대자동차 2023년도 재무정보도 조회해보기
hyundai_corp_code = get_corp_code("현대자동차", corp_df)
if hyundai_corp_code:
    print("\n" + "="*60)
    hyundai_fs_2023 = get_financial_statement(hyundai_corp_code, "2023", "11011")



💰 재무정보 조회 중... (2023년 사업보고서(연간))
✅ 2023년 사업보고서(연간) 재무 정보 (30개 항목)

📊 재무정보 요약 (상위 15개 항목):
✅ 2023년 사업보고서(연간) 재무 정보 (30개 항목)

📊 재무정보 요약 (상위 15개 항목):


,sj_nm,account_nm,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount
0,재무상태표,유동자산,제 55 기,"195,936,557,000,000",제 54 기,"218,470,581,000,000"
1,재무상태표,비유동자산,제 55 기,"259,969,423,000,000",제 54 기,"229,953,926,000,000"
2,재무상태표,자산총계,제 55 기,"455,905,980,000,000",제 54 기,"448,424,507,000,000"
3,재무상태표,유동부채,제 55 기,"75,719,452,000,000",제 54 기,"78,344,852,000,000"
4,재무상태표,비유동부채,제 55 기,"16,508,663,000,000",제 54 기,"15,330,051,000,000"
5,재무상태표,부채총계,제 55 기,"92,228,115,000,000",제 54 기,"93,674,903,000,000"
6,재무상태표,자본금,제 55 기,"897,514,000,000",제 54 기,"897,514,000,000"
7,재무상태표,이익잉여금,제 55 기,"346,652,238,000,000",제 54 기,"337,946,407,000,000"
8,재무상태표,자본총계,제 55 기,"363,677,865,000,000",제 54 기,"354,749,604,000,000"
9,손익계산서,매출액,제 55 기,"258,935,494,000,000",제 54 기,"302,231,360,000,000"



💡 주요 재무지표:
   📈 매출액: 258,935,494,000,000
   📈 영업이익: 6,566,976,000,000
   📈 당기순이익: 15,487,100,000,000
   📈 자본총계: 363,677,865,000,000
🔍 '현대자동차'와 일치하는 기업이 5개 있습니다:
   - 현대자동차써비스 (코드: 00164751, 종목: None)
   - 정왕점현대자동차 (코드: 01333297, 종목: None)
   - 현대자동차충청써비스 (코드: 00914688, 종목: None)
   - 수원현대자동차운전전문학원 (코드: 01370216, 종목: None)
   - 현대자동차 (코드: 00164742, 종목: 005380)
✅ 정확히 일치하는 기업 선택: 현대자동차

💰 재무정보 조회 중... (2023년 사업보고서(연간))
✅ 2023년 사업보고서(연간) 재무 정보 (30개 항목)

📊 재무정보 요약 (상위 15개 항목):
✅ 2023년 사업보고서(연간) 재무 정보 (30개 항목)

📊 재무정보 요약 (상위 15개 항목):


,sj_nm,account_nm,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount
0,재무상태표,유동자산,제 56 기,"101,724,717,000,000",제 55 기,"96,389,273,000,000"
1,재무상태표,비유동자산,제 56 기,"180,738,638,000,000",제 55 기,"159,353,189,000,000"
2,재무상태표,자산총계,제 56 기,"282,463,355,000,000",제 55 기,"255,742,462,000,000"
3,재무상태표,유동부채,제 56 기,"73,362,103,000,000",제 55 기,"74,236,472,000,000"
4,재무상태표,비유동부채,제 56 기,"107,291,812,000,000",제 55 기,"90,609,445,000,000"
5,재무상태표,부채총계,제 56 기,"180,653,915,000,000",제 55 기,"164,845,917,000,000"
6,재무상태표,자본금,제 56 기,"1,488,993,000,000",제 55 기,"1,488,993,000,000"
7,재무상태표,이익잉여금,제 56 기,"88,665,805,000,000",제 55 기,"79,953,601,000,000"
8,재무상태표,자본총계,제 56 기,"101,809,440,000,000",제 55 기,"90,896,545,000,000"
9,손익계산서,매출액,제 56 기,"162,663,579,000,000",제 55 기,"142,151,469,000,000"



💡 주요 재무지표:
   📈 매출액: 162,663,579,000,000
   📈 영업이익: 15,126,901,000,000
   📈 당기순이익: 12,272,301,000,000
   📈 자본총계: 101,809,440,000,000


## 5. 대주주 현황 조회 (majorstockholders.json)

기업의 대주주(최대주주 및 주요주주)에 대한 정보를 조회합니다.

- **API Endpoint**: `/majorstockholders.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호

**함수 동작:**
1. `corp_code` 유효성 확인
2. 짧은 지연(0.5초) 후 API 호출
3. 응답 JSON 처리 및 오류 검증
4. 조회된 주주 목록을 DataFrame으로 변환하고 주요 컬럼(주주명, 보유주식수, 지분율 등)을 테이블로 표시

**함수 정의:**
```python
get_major_stockholders(corp_code)
```

**사용 예시:**
```python
# 삼성전자 대주주 현황 조회
major_holders = get_major_stockholders(samsung_corp_code)
```  

In [7]:
## 7. 최근 공시 목록 조회 (수정된 버전)

def get_recent_disclosures(corp_code, bgn_de=None, end_de=None, pblntf_ty=None, page_no=1, page_count=10):
    """기업의 최근 공시 목록을 조회합니다."""
    if not corp_code:
        print("❌ 기업 고유번호가 없어 조회할 수 없습니다.")
        return None
    
    print(f"📋 공시 목록 조회 중... (최근 {page_count}건)")
    
    url = f"{BASE_URL}/list.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'page_no': page_no,
        'page_count': page_count
    }
    
    if bgn_de: 
        params['bgn_de'] = bgn_de
        print(f"   📅 시작일: {bgn_de}")
    if end_de: 
        params['end_de'] = end_de
        print(f"   📅 종료일: {end_de}")
    if pblntf_ty: 
        params['pblntf_ty'] = pblntf_ty
        pblntf_names = {'A': '정기공시', 'B': '주요사항보고', 'C': '발행공시'}
        print(f"   📑 공시유형: {pblntf_names.get(pblntf_ty, pblntf_ty)}")
    
    # API 호출 제한을 고려한 지연
    time.sleep(0.5)
    
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        disclosures = data['list']
        total_count = data.get('total_count', len(disclosures))
        total_page = data.get('total_page', 1)
        current_page = data.get('page_no', page_no)
        
        print(f"✅ 공시 목록 조회 완료")
        print(f"   📊 총 {total_count:,}건 중 {len(disclosures)}건 표시")
        print(f"   📄 페이지: {current_page}/{total_page}")
        
        df = pd.DataFrame(disclosures)
        
        # 주요 컬럼만 선택하여 표시
        display_columns = ['rcept_dt', 'corp_name', 'report_nm', 'flr_nm', 'rcept_no']
        available_columns = [col for col in display_columns if col in df.columns]
        
        if available_columns:
            print(f"\n📋 공시 목록:")
            display_df = df[available_columns].copy()
            
            # 날짜 형식 개선
            if 'rcept_dt' in display_df.columns:
                display_df['rcept_dt'] = pd.to_datetime(display_df['rcept_dt']).dt.strftime('%Y-%m-%d')
            
            display(display_df)
            
            # 공시 원문 URL 예시 (첫 번째 공시만)
            if len(disclosures) > 0:
                first_disclosure = disclosures[0]
                rcept_no = first_disclosure.get('rcept_no')
                viewer_url = f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}"
                print(f"\n🔗 첫 번째 공시 원문 보기:")
                display(Markdown(f"[{first_disclosure.get('report_nm', 'N/A')} 원문보기]({viewer_url})"))
        
        return disclosures
    else:
        print("❌ 공시 목록을 찾을 수 없습니다.")
        print("   💡 검색 조건을 변경해보세요.")
    return None

# 삼성전자 최근 공시 조회 테스트
from datetime import datetime, timedelta

today = datetime.today().strftime('%Y%m%d')
one_month_ago = (datetime.today() - timedelta(days=30)).strftime('%Y%m%d')

if samsung_corp_code:
    print("\n" + "="*60)
    samsung_disclosures = get_recent_disclosures(
        samsung_corp_code, 
        bgn_de=one_month_ago, 
        end_de=today, 
        page_count=5
    )



📋 공시 목록 조회 중... (최근 5건)
   📅 시작일: 20250505
   📅 종료일: 20250604
✅ 공시 목록 조회 완료
   📊 총 14건 중 5건 표시
   📄 페이지: 1/3

📋 공시 목록:
✅ 공시 목록 조회 완료
   📊 총 14건 중 5건 표시
   📄 페이지: 1/3

📋 공시 목록:


,rcept_dt,corp_name,report_nm,flr_nm,rcept_no
0,2025-06-02,삼성전자,대규모기업집단현황공시[연1회(동일인용)],이재용,20250602000137
1,2025-06-02,삼성전자,대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)],삼성전자,20250602000112
2,2025-05-30,삼성전자,기업지배구조보고서공시,삼성전자,20250530801005
3,2025-05-28,삼성전자,자기주식처분결과보고서,삼성전자,20250528000650
4,2025-05-23,삼성전자,주식등의대량보유상황보고서(일반),삼성물산,20250523000612



🔗 첫 번째 공시 원문 보기:


[대규모기업집단현황공시[연1회(동일인용)] 원문보기](http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20250602000137)

## 6. 최근 공시 목록 조회 (list.json)

특정 기업의 최근 공시된 문서 목록을 조회합니다. 다양한 필터 옵션을 활용하여 원하는 공시 정보를 세밀하게 조회할 수 있습니다.

- **API Endpoint**: `/list.json`
- **필수 파라미터**:
  - `crtfc_key`: API 인증키
  - `corp_code`: 기업 고유번호
- **선택 파라미터**:
  - `bgn_de` (YYYYMMDD): 검색 시작일
  - `end_de` (YYYYMMDD): 검색 종료일
  - `pblntf_ty` (문자): 공시유형 (A: 정기공시, B: 주요사항보고, C: 발행공시 등)
  - `page_no` (정수, 기본값: 1): 페이지 번호
  - `page_count` (정수, 기본값: 10, 최대값: 100): 페이지당 결과 수

**함수 정의**:
```python
get_recent_disclosures(corp_code, bgn_de=None, end_de=None, pblntf_ty=None, page_no=1, page_count=10)
```

**함수 동작 흐름**:
1. `corp_code` 유효성 검사 (없을 경우 즉시 종료)
2. 조회 시작 메시지 및 전달된 파라미터 출력
3. 0.5초 지연 후 API 호출
4. 응답 처리 및 오류/데이터 없음 핸들링(`handle_api_response`)
5. 응답 JSON의 `list` 항목을 DataFrame으로 변환
6. 주요 컬럼(`rcept_dt`, `corp_name`, `report_nm`, `flr_nm`, `rcept_no`)을 선택하여 테이블 표시
7. 총 건수, 페이지 정보 출력
8. 첫 번째 공시의 원문 URL 예시 제공 (선택 사항)

**사용 예시**:
```python
from datetime import datetime, timedelta

today = datetime.today().strftime('%Y%m%d')
one_month_ago = (datetime.today() - timedelta(days=30)).strftime('%Y%m%d')

recent = get_recent_disclosures(
    samsung_corp_code,
    bgn_de=one_month_ago,
    end_de=today,
    page_count=5
)
```  

In [8]:
## 7. 최근 공시 목록 조회 (수정된 버전)

def get_recent_disclosures(corp_code, bgn_de=None, end_de=None, pblntf_ty=None, page_no=1, page_count=10):
    """기업의 최근 공시 목록을 조회합니다."""
    if not corp_code:
        print("❌ 기업 고유번호가 없어 조회할 수 없습니다.")
        return None
    
    print(f"📋 공시 목록 조회 중... (최근 {page_count}건)")
    
    url = f"{BASE_URL}/list.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'page_no': page_no,
        'page_count': page_count
    }
    
    if bgn_de: 
        params['bgn_de'] = bgn_de
        print(f"   📅 시작일: {bgn_de}")
    if end_de: 
        params['end_de'] = end_de
        print(f"   📅 종료일: {end_de}")
    if pblntf_ty: 
        params['pblntf_ty'] = pblntf_ty
        pblntf_names = {'A': '정기공시', 'B': '주요사항보고', 'C': '발행공시'}
        print(f"   📑 공시유형: {pblntf_names.get(pblntf_ty, pblntf_ty)}")
    
    # API 호출 제한을 고려한 지연
    time.sleep(0.5)
    
    response = requests.get(url, params=params)
    data = handle_api_response(response)
    
    if data and data.get('list'):
        disclosures = data['list']
        total_count = data.get('total_count', len(disclosures))
        total_page = data.get('total_page', 1)
        current_page = data.get('page_no', page_no)
        
        print(f"✅ 공시 목록 조회 완료")
        print(f"   📊 총 {total_count:,}건 중 {len(disclosures)}건 표시")
        print(f"   📄 페이지: {current_page}/{total_page}")
        
        df = pd.DataFrame(disclosures)
        
        # 주요 컬럼만 선택하여 표시
        display_columns = ['rcept_dt', 'corp_name', 'report_nm', 'flr_nm', 'rcept_no']
        available_columns = [col for col in display_columns if col in df.columns]
        
        if available_columns:
            print(f"\n📋 공시 목록:")
            display_df = df[available_columns].copy()
            
            # 날짜 형식 개선
            if 'rcept_dt' in display_df.columns:
                display_df['rcept_dt'] = pd.to_datetime(display_df['rcept_dt']).dt.strftime('%Y-%m-%d')
            
            display(display_df)
            
            # 공시 원문 URL 예시 (첫 번째 공시만)
            if len(disclosures) > 0:
                first_disclosure = disclosures[0]
                rcept_no = first_disclosure.get('rcept_no')
                viewer_url = f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}"
                print(f"\n🔗 첫 번째 공시 원문 보기:")
                display(Markdown(f"[{first_disclosure.get('report_nm', 'N/A')} 원문보기]({viewer_url})"))
        
        return disclosures
    else:
        print("❌ 공시 목록을 찾을 수 없습니다.")
        print("   💡 검색 조건을 변경해보세요.")
    return None

# 삼성전자 최근 공시 조회 테스트
from datetime import datetime, timedelta

today = datetime.today().strftime('%Y%m%d')
one_month_ago = (datetime.today() - timedelta(days=30)).strftime('%Y%m%d')

if samsung_corp_code:
    print("\n" + "="*60)
    samsung_disclosures = get_recent_disclosures(
        samsung_corp_code, 
        bgn_de=one_month_ago, 
        end_de=today, 
        page_count=5
    )



📋 공시 목록 조회 중... (최근 5건)
   📅 시작일: 20250505
   📅 종료일: 20250604
✅ 공시 목록 조회 완료
   📊 총 14건 중 5건 표시
   📄 페이지: 1/3

📋 공시 목록:
✅ 공시 목록 조회 완료
   📊 총 14건 중 5건 표시
   📄 페이지: 1/3

📋 공시 목록:


,rcept_dt,corp_name,report_nm,flr_nm,rcept_no
0,2025-06-02,삼성전자,대규모기업집단현황공시[연1회(동일인용)],이재용,20250602000137
1,2025-06-02,삼성전자,대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)],삼성전자,20250602000112
2,2025-05-30,삼성전자,기업지배구조보고서공시,삼성전자,20250530801005
3,2025-05-28,삼성전자,자기주식처분결과보고서,삼성전자,20250528000650
4,2025-05-23,삼성전자,주식등의대량보유상황보고서(일반),삼성물산,20250523000612



🔗 첫 번째 공시 원문 보기:


[대규모기업집단현황공시[연1회(동일인용)] 원문보기](http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20250602000137)

## 7. 통합 기업 정보 조회 함수

이 섹션에서는 회사명을 입력받아 기업 고유번호 조회, 개황 정보 조회, 재무정보 조회, 최근 공시 목록 조회를
순차적으로 수행하고, 모든 결과를 사전 형태로 반환하는 통합 함수 `get_comprehensive_company_info`를 정의합니다.

**주요 기능:**
- 기업 고유번호(corp_code) 자동 조회
- 기업 개황(overview) 정보 조회 및 예외 처리
- 지정 연도(year)의 재무정보(financial) 조회 및 예외 처리
- 최근 공시 목록(disclosures) 조회 및 예외 처리

**매개변수:**
- `company_name` (str): 정보를 조회할 기업명
- `year` (str, 기본값 "2023"): 조회할 연도

**반환값:**
- dict: {
  - `corp_code`: 조회된 기업 고유번호,
  - `overview`: 개황 정보 데이터(dict 혹은 None),
  - `financial`: 재무정보 리스트(list 혹은 None),
  - `disclosures`: 공시 목록 리스트(list 혹은 None)
}

**사용 예시:**
```python
comprehensive_info = get_comprehensive_company_info("카카오", "2023")
print(comprehensive_info)
```

In [9]:
## 8. 통합 기업 정보 조회 함수

def get_comprehensive_company_info(company_name, year="2023"):
    """기업명을 입력받아 종합적인 기업 정보를 조회하는 함수"""
    print(f"🔍 '{company_name}' 종합 정보 조회를 시작합니다...")
    print("="*60)
    
    # 1. 기업 고유번호 조회
    corp_code = get_corp_code(company_name, corp_df)
    if not corp_code:
        print(f"❌ '{company_name}' 기업을 찾을 수 없습니다.")
        return None
    
    results = {'corp_code': corp_code}
    
    # 2. 기업 개황 정보
    print(f"\n📊 1단계: 기업 개황 정보 조회")
    try:
        company_overview = get_company_overview(corp_code)
        results['overview'] = company_overview
    except Exception as e:
        print(f"❌ 기업 개황 조회 실패: {e}")
        results['overview'] = None
    
    # 3. 재무정보 조회
    print(f"\n💰 2단계: {year}년 재무정보 조회")
    try:
        financial_info = get_financial_statement(corp_code, year, "11011")
        results['financial'] = financial_info
    except Exception as e:
        print(f"❌ 재무정보 조회 실패: {e}")
        results['financial'] = None
    
    # 4. 최근 공시 조회
    print(f"\n📋 3단계: 최근 공시 목록 조회")
    try:
        today = datetime.today().strftime('%Y%m%d')
        three_months_ago = (datetime.today() - timedelta(days=90)).strftime('%Y%m%d')
        
        recent_disclosures = get_recent_disclosures(
            corp_code, 
            bgn_de=three_months_ago, 
            end_de=today, 
            page_count=10
        )
        results['disclosures'] = recent_disclosures
    except Exception as e:
        print(f"❌ 공시 목록 조회 실패: {e}")
        results['disclosures'] = None
    
    print(f"\n✅ '{company_name}' 종합 정보 조회가 완료되었습니다!")
    return results

# 종합 조회 테스트
print("🧪 종합 기업 정보 조회 테스트")
test_company = "카카오"
comprehensive_info = get_comprehensive_company_info(test_company, "2023")

🧪 종합 기업 정보 조회 테스트
🔍 '카카오' 종합 정보 조회를 시작합니다...
🔍 '카카오'와 일치하는 기업이 27개 있습니다:
   - 카카오 (코드: 00918444, 종목: None)
   - 카카오랩 (코드: 01143205, 종목: None)
   - 카카오게임즈홀딩스 (코드: 01144329, 종목: None)
   - 카카오엠 (코드: 00130949, 종목: 016170)
   - 아산카카오메디컬데이터 (코드: 01359569, 종목: None)
   - 카카오메이커스 (코드: 01248795, 종목: None)
   - 카카오엠 (코드: 01337460, 종목: None)
   - 카카오커머스 (코드: 01351062, 종목: None)
   - 카카오모빌리티 (코드: 01250666, 종목: None)
   - 카카오파트너 (코드: 01687985, 종목: None)
✅ 정확히 일치하는 기업 선택: 카카오

📊 1단계: 기업 개황 정보 조회
📊 기업 개황 정보 조회 중... (고유번호: 00918444)

🏢 [(주)카카오] 기업 개황 정보
🏢 기업명: (주)카카오
🌐 영문명: Kakao.Corp
👨‍💼 대표자: 이제범,이석우
🏠 주소: 서울특별시 강남구 역삼동 726-2 C&K빌딩 2층
📞 전화번호: 070-7492-1305
📅 설립일: 20061201
📊 결산월: 12

📋 요약 테이블:

🏢 [(주)카카오] 기업 개황 정보
🏢 기업명: (주)카카오
🌐 영문명: Kakao.Corp
👨‍💼 대표자: 이제범,이석우
🏠 주소: 서울특별시 강남구 역삼동 726-2 C&K빌딩 2층
📞 전화번호: 070-7492-1305
📅 설립일: 20061201
📊 결산월: 12

📋 요약 테이블:


,corp_name,stock_code,ceo_nm,adres,est_dt
0,(주)카카오,,"이제범,이석우",서울특별시 강남구 역삼동 726-2 C&K빌딩 2층,20061201



💰 2단계: 2023년 재무정보 조회
💰 재무정보 조회 중... (2023년 사업보고서(연간))
📭 데이터 없음: 조회된 데이타가 없습니다.
❌ 2023년 사업보고서(연간) 재무 정보를 찾을 수 없습니다.
   💡 다른 연도나 보고서 유형을 시도해보세요.

📋 3단계: 최근 공시 목록 조회
📋 공시 목록 조회 중... (최근 10건)
   📅 시작일: 20250306
   📅 종료일: 20250604
📭 데이터 없음: 조회된 데이타가 없습니다.
❌ 2023년 사업보고서(연간) 재무 정보를 찾을 수 없습니다.
   💡 다른 연도나 보고서 유형을 시도해보세요.

📋 3단계: 최근 공시 목록 조회
📋 공시 목록 조회 중... (최근 10건)
   📅 시작일: 20250306
   📅 종료일: 20250604
📭 데이터 없음: 조회된 데이타가 없습니다.
❌ 공시 목록을 찾을 수 없습니다.
   💡 검색 조건을 변경해보세요.

✅ '카카오' 종합 정보 조회가 완료되었습니다!
📭 데이터 없음: 조회된 데이타가 없습니다.
❌ 공시 목록을 찾을 수 없습니다.
   💡 검색 조건을 변경해보세요.

✅ '카카오' 종합 정보 조회가 완료되었습니다!


## 8. 결론 및 사용 가이드

이 노트북을 통해 OpenDART API의 주요 기능을 테스트하여 AI Agent 개발 시 활용할 수 있는 기업 정보 수집 워크플로우를 확인했습니다.

**수집 가능한 주요 정보:**
- 기업 고유번호
- 기업 일반 현황 (회사명, 주소, 대표자, 사업목적 등)
- 재무제표 (자산, 부채, 자본, 손익계산서 주요 항목 등)
- 대주주 및 지분 현황
- 최근 공시 목록 및 공시 원문 접근 정보

**사용 가능한 주요 함수:**

1️⃣ `get_corp_code(company_name)`  
2️⃣ `get_company_overview(corp_code)`  
3️⃣ `get_financial_statement(corp_code, year, report_code)`  
4️⃣ `get_recent_disclosures(corp_code, start_date, end_date, page_count)`  
5️⃣ `get_comprehensive_company_info(company_name, year)`  

**사용 예시:**
```python
company_info = get_comprehensive_company_info("네이버", "2023")
```  

**주의사항:**
- API 호출 제한이 있으므로 과도한 요청을 피하세요.  
- 일부 기업의 경우 특정 연도 데이터가 없을 수 있습니다.  
- API 키는 안전하게 관리하세요.

**추가 테스트 가능한 주요 기업 리스트:**
```python
major_companies = ["삼성전자", "SK하이닉스", "NAVER", "카카오", "LG화학", "현대자동차", "기아", "포스코홀딩스"]
```  

**테스트 호출 예시:**
```python
get_comprehensive_company_info("기업명", "2023")
```

In [10]:
## 9. 결론 및 사용 가이드

print("\n" + "="*80)
print("🎉 OpenDART API 테스트 완료!")
print("="*80)

print("""
📋 사용 가능한 주요 함수들:

1️⃣ get_corp_code(company_name) 
   - 회사명으로 기업 고유번호 조회

2️⃣ get_company_overview(corp_code)
   - 기업 개황 정보 조회 (기본 정보, 대표자, 주소 등)

3️⃣ get_financial_statement(corp_code, year, report_code)
   - 재무제표 조회
   - report_code: "11011"(사업보고서), "11012"(반기), "11013"(1분기), "11014"(3분기)

4️⃣ get_recent_disclosures(corp_code, start_date, end_date, page_count)
   - 공시 목록 조회

5️⃣ get_comprehensive_company_info(company_name, year)
   - 종합 기업 정보 조회 (위의 모든 정보를 한번에)

💡 사용 예시:
   company_info = get_comprehensive_company_info("네이버", "2023")

⚠️ 주의사항:
   - API 호출 제한이 있으므로 과도한 요청은 피해주세요
   - 일부 기업의 경우 특정 연도 데이터가 없을 수 있습니다
   - API 키는 안전하게 관리하세요
""")

# 추가 테스트를 위한 기업 리스트
print("\n🏢 추가 테스트 가능한 주요 기업들:")
major_companies = ["삼성전자", "SK하이닉스", "NAVER", "카카오", "LG화학", "현대자동차", "기아", "포스코홀딩스"]
for i, company in enumerate(major_companies, 1):
    print(f"   {i}. {company}")

print(f"\n💡 테스트하려면: get_comprehensive_company_info('기업명', '2023')")


🎉 OpenDART API 테스트 완료!

📋 사용 가능한 주요 함수들:

1️⃣ get_corp_code(company_name) 
   - 회사명으로 기업 고유번호 조회

2️⃣ get_company_overview(corp_code)
   - 기업 개황 정보 조회 (기본 정보, 대표자, 주소 등)

3️⃣ get_financial_statement(corp_code, year, report_code)
   - 재무제표 조회
   - report_code: "11011"(사업보고서), "11012"(반기), "11013"(1분기), "11014"(3분기)

4️⃣ get_recent_disclosures(corp_code, start_date, end_date, page_count)
   - 공시 목록 조회

5️⃣ get_comprehensive_company_info(company_name, year)
   - 종합 기업 정보 조회 (위의 모든 정보를 한번에)

💡 사용 예시:
   company_info = get_comprehensive_company_info("네이버", "2023")

⚠️ 주의사항:
   - API 호출 제한이 있으므로 과도한 요청은 피해주세요
   - 일부 기업의 경우 특정 연도 데이터가 없을 수 있습니다
   - API 키는 안전하게 관리하세요


🏢 추가 테스트 가능한 주요 기업들:
   1. 삼성전자
   2. SK하이닉스
   3. NAVER
   4. 카카오
   5. LG화학
   6. 현대자동차
   7. 기아
   8. 포스코홀딩스

💡 테스트하려면: get_comprehensive_company_info('기업명', '2023')
